## 사용 패키지

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'

# from sklearn.ensemble import Rando|mForestRegressor
# import lightgbm as lgb
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

# `0704 추가`

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [3]:
# HYPER PARAMETERS

class CFG:
    emb_dim = 200

args = CFG

df_train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
df_test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')

#word embedding

df_all = pd.concat([df_train, df_test]) ## 여기 임의수정. df -> df.train

# Basic text preprocessing

def split_process(x, q):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        if '/' in i:
            x_.extend(i.split('/'))
        else:
            x_.append(i)
    x_ = list(set(x_))
    x_.remove('')
    return x_

# Get all combinations for training w2v (train + test)

food_combinations = []
for i in ['조식메뉴', '중식메뉴', '석식메뉴']:
    food_combinations += df_all[i].apply(lambda x: split_process(x, i)).to_list()
    
# Train or load w2v model

TRAIN_W2V = True
try:
    model = Word2Vec.load('food_embedding.model')
    print("Model loaded")
except:
    if TRAIN_W2V:
        print("Training w2v")
        model = Word2Vec(sentences=food_combinations, vector_size=args.emb_dim, window=7, min_count=0, workers=4, sg=0, epochs=5000)
        model.save('food_embedding.model')
    else:
        print("Model loading failed. Do not train.")
        
# w2v demo
model.wv.most_similar('된장찌개')

Training w2v


TypeError: __init__() got an unexpected keyword argument 'vector_size'

In [ ]:
def process_date(df):
    df['일자'] = pd.to_datetime(df['일자'], format="%Y-%m-%d")
    df['year'] = df['일자'].dt.year
    df['month'] = df['일자'].dt.month
    df['day'] = df['일자'].dt.day
    df = df.drop('일자', axis=1)
    return df

In [ ]:
def get_food_embedding(x):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        if '/' in i:
            x_.extend(i.split('/'))
        else:
            x_.append(i)
    x_ = list(set(x_))
    x_.remove('')
    vec_ = np.zeros(args.emb_dim)
    for i in x_:
        vec = model.wv.get_vector(i)
        vec_ += vec
    vec_ /= len(x_)
    return vec_

# General preprocessing
df_train = process_date(df_train)
day_encoder = LabelEncoder()
df_train['요일'] = day_encoder.fit_transform(df_train['요일'])

# Get embedding
df_train['조식메뉴_embedding'] = df_train['조식메뉴'].apply(lambda x: get_food_embedding(x))
df_train['중식메뉴_embedding'] = df_train['중식메뉴'].apply(lambda x: get_food_embedding(x))
df_train['석식메뉴_embedding'] = df_train['석식메뉴'].apply(lambda x: get_food_embedding(x))

y_lunch = df_train['중식계']
y_dinner = df_train['석식계']
df_train.drop(['조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계'], axis=1, inplace=True)

X_common = df_train.iloc[:, :9]

# emb_arr_lunch = np.array(df_train.iloc[:, 9].to_numpy().tolist()) + np.array(df_train.iloc[:, 10].to_numpy().tolist()) # Ver 1
# emb_arr_dinner = np.array(df_train.iloc[:, 9].to_numpy().tolist()) + np.array(df_train.iloc[:, 11].to_numpy().tolist()) # Ver 1
emb_arr_lunch = np.array(df_train.iloc[:, 10].to_numpy().tolist()) # Ver 2
emb_arr_dinner = np.array(df_train.iloc[:, 11].to_numpy().tolist()) # Ver 2

X_train_lunch = np.concatenate((X_common.to_numpy(), emb_arr_lunch), axis=1)
X_train_dinner = np.concatenate((X_common.to_numpy(), emb_arr_dinner), axis=1)

X_train_lunch, X_test_lunch, y_train_lunch, y_test_lunch = train_test_split(X_train_lunch, y_lunch, test_size=0.1, random_state=42)
X_train_dinner, X_test_dinner, y_train_dinner, y_test_dinner = train_test_split(X_train_dinner, y_dinner, test_size=0.1, random_state=42)

# Simple LGBM Regressor w/o tuning
model_lunch = LGBMRegressor()
model_lunch.fit(X_train_lunch, y_train_lunch)

model_dinner = LGBMRegressor()
model_dinner.fit(X_train_dinner, y_train_dinner)

# Validate
pred_lunch = model_lunch.predict(X_test_lunch)
pred_dinner = model_dinner.predict(X_test_dinner)

print("lunch mae: ", mean_absolute_error(y_test_lunch, pred_lunch))
print("dinner mae: ", mean_absolute_error(y_test_dinner, pred_dinner))

inference

In [ ]:
df_test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')

# Apply general preprocessing
df_test= process_date(df_test)
df_test['요일'] = day_encoder.transform(df_test['요일'])
df_test['조식메뉴_embedding'] = df_test['조식메뉴'].apply(lambda x: get_food_embedding(x))
df_test['중식메뉴_embedding'] = df_test['중식메뉴'].apply(lambda x: get_food_embedding(x))
df_test['석식메뉴_embedding'] = df_test['석식메뉴'].apply(lambda x: get_food_embedding(x))
df_test.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis=1, inplace=True)
X_test_common = df_test.iloc[:, :9]

emb_arr_lunch = np.array(df_train.iloc[:, 10].to_numpy().tolist()) # Ver 2
emb_arr_dinner = np.array(df_train.iloc[:, 11].to_numpy().tolist()) # Ver 2

# Get embedding
test_emb_arr_lunch = np.array(df_test.iloc[:, 10].to_numpy().tolist()) # Ver 2
test_emb_arr_dinner = np.array(df_test.iloc[:, 11].to_numpy().tolist()) # Ver 2
# Concat
test_lunch = np.concatenate((X_test_common.to_numpy(), test_emb_arr_lunch), axis=1)
test_dinner = np.concatenate((X_test_common.to_numpy(), test_emb_arr_dinner), axis=1)

# Inference

test_pred_lunch = model_lunch.predict(test_lunch)
test_pred_dinner = model_dinner.predict(test_dinner)

submission_df = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission_df['중식계'] = test_pred_lunch
submission_df['석식계'] = test_pred_dinner

# Save

submission_df.to_csv('sub_2.csv', index=False)

---

# `0705 추가 - 메뉴구분`

In [ ]:
from tqdm.notebook import tqdm
from pycaret.regression import *

In [4]:
def get_food_embedding(x):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        if '/' in i:
            x_.extend(i.split('/'))
        else:
            x_.append(i)
    x_ = list(set(x_))
    x_.remove('')
    return x_

train['중식메뉴_split'] = train['중식메뉴'].apply(lambda x: get_food_embedding(x))
train['석식메뉴_split'] = train['석식메뉴'].apply(lambda x: get_food_embedding(x))

NameError: name 'train' is not defined

In [26]:
test['중식메뉴_split'] = test['중식메뉴'].apply(lambda x: get_food_embedding(x))
test['석식메뉴_split'] = test['석식메뉴'].apply(lambda x: get_food_embedding(x))

In [27]:
train['중식메뉴_split'].shape

(1205,)

In [33]:
train['중식메뉴_split'][1]

['쌀밥', '가자미튀김', '모둠소세지구이', '마늘쫑무침', '잡곡밥', '배추겉절이', '김치찌개', '요구르트']

In [16]:
lunch_lst = []
lunch_count = []
pref = 0
for i in tqdm(range(1205)):
  try:
    if train['중식계'][i] >= 880:
      pref = 1
    else:
      pref = 0

    lunch_lst.append(train['중식메뉴_split'][i])
    lunch_count.append(pref)
  except:
    pass
# for i in range(len(lunch_lst)):
  # lunch_lst[i] = lunch_lst[i].replace(',', ' ')
print('done')

  0%|          | 0/1205 [00:00<?, ?it/s]

done


In [25]:
dinner_lst = []
dinner_count = []
pref_d = 0
for i in tqdm(range(1205)):
  try:
    if train['석식계'][i] > 476:
      pref_d = 1
    else:
      pref_d = 0

    dinner_lst.append(train['석식메뉴_split'][i])
    dinner_count.append(pref_d)
  except:
    pass
# for i in range(len(dinner_lst)):
#   dinner_lst[i] = dinner_lst[i].replace(',', ' ')
print('done')

  0%|          | 0/1205 [00:00<?, ?it/s]

done


In [26]:
lunch_lst_test = []
lunch_count_test = []
for i in tqdm(range(50)):
  try:
    lunch_lst_test.append(test['중식메뉴_split'][i])
    lunch_count_test.append(0)
  except:
    pass
# for i in range(len(lunch_lst_test)):
#   lunch_lst_test[i] = lunch_lst_test[i].replace(',', ' ')
print('done')

  0%|          | 0/50 [00:00<?, ?it/s]

done


In [27]:
dinner_lst_test = []
dinner_count_test = []
for i in tqdm(range(50)):
  try:
    dinner_lst_test.append(test['석식메뉴_split'][i])
    dinner_count_test.append(0)
  except:
    pass
# for i in range(len(dinner_lst_test)):
#   dinner_lst_test[i] = dinner_lst_test[i].replace(',', ' ')
print('done')

  0%|          | 0/50 [00:00<?, ?it/s]

done


In [34]:
lunch_df = pd.DataFrame({'중식메뉴':lunch_lst})
dinner_df = pd.DataFrame({'석식메뉴':dinner_lst})
lunch_df_t = pd.DataFrame({'중식메뉴':lunch_lst_test})
dinner_df_t = pd.DataFrame({'석식메뉴':dinner_lst_test})

In [38]:
# -----------------------------<Full DATA>-----------------------------------

train_lunch = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월', '일','주','년','중식계']]
test_lunch = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월', '일','주','년']]

train_dinner = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월', '일','주','년','석식계']]
test_dinner = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월', '일','주','년']]

In [39]:
train_lunch

,요일,본사시간외근무명령서승인건수,식사가능자수,월,일,주,년,중식계
0,0,238,2551.0,2,1,5,2016,1039.0
1,1,319,2551.0,2,2,5,2016,867.0
2,2,111,2545.0,2,3,5,2016,1017.0
3,3,355,2497.0,2,4,5,2016,978.0
4,4,34,2323.0,2,5,5,2016,925.0
...,...,...,...,...,...,...,...,...
1200,2,4,2517.0,1,20,3,2021,1093.0
1201,3,462,2540.0,1,21,3,2021,832.0
1202,4,1,2425.0,1,22,3,2021,579.0
1203,0,616,2549.0,1,25,4,2021,1145.0


In [40]:
#메뉴
train_lunch = pd.concat([train_lunch, lunch_sort], axis = 1)
train_dinner = pd.concat([train_dinner, dinner_sort], axis = 1)
test_lunch = pd.concat([test_lunch, lunch_sort_t], axis = 1)
test_dinner = pd.concat([test_dinner, dinner_sort_t], axis = 1)
print('done')

NameError: name 'lunch_sort' is not defined

# `0707 추가 메뉴구분2 ( 간단한 EDA )`

### train 점심 전처리

In [6]:
# 일별 점심메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch) 만들기  TRAIN
lunch1 = []
for day in range(len(train)):
    tmp = train.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
#    search = ')'   # 원산지 정보는 삭제
#    for menu in tmp:
#        if search in menu:
#            tmp.remove(menu)
    
    lunch1.append(tmp) 

### test 점심 전처리

In [7]:
# 일별 점심메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch) 만들기  TRAIN
lunch2 = []
for day in range(len(test)):
    tmp = test.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    search = ')'   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch2.append(tmp) 

### train 점심 메뉴 구분

In [8]:
# lunch train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = []
for i, day_menu in enumerate(lunch1):
#    bob_tmp = day_menu[0]; bob.append(bob_tmp)
    gook_tmp = day_menu[1]; gook.append(gook_tmp)
    banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
    banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
    banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

    if i < 1067:
#        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
        side_tmp = day_menu[-2]; side.append(side_tmp)
    else: 
#        kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
        side_tmp  = day_menu[-1]; side.append(side_tmp)

### 점심 train set 제작

In [9]:
train_ln = x1_train
#train_ln['bob'] = bob
train_ln['gook'] = gook
train_ln['banchan1'] = banchan1
train_ln['banchan2'] = banchan2 
train_ln['banchan3'] = banchan3
#train_ln['kimchi'] = kimchi 
train_ln['side'] = side

#train_ln['bob'] = train_ln['bob'].astype(object)
#train_ln['gook'] = train_ln['gook'].astype(object)
#train_ln['banchan1'] = train_ln['banchan1'].astype(object)
#train_ln['banchan2'] = train_ln['banchan2'].astype(object)
#train_ln['banchan3'] = train_ln['banchan3'].astype(object)
#train_ln['kimchi'] = train_ln['kimchi'].astype(object)
#train_ln['side'] = train_ln['side'].astype(object)

<ipython-input-9-8811194a5d6c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ln['gook'] = gook
<ipython-input-9-8811194a5d6c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ln['banchan1'] = banchan1
<ipython-input-9-8811194a5d6c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

### test 점심 메뉴 구분

In [10]:
# lunch train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = []
for i, day_menu in enumerate(lunch2):
#    bob_tmp = day_menu[0]; bob.append(bob_tmp)
    gook_tmp = day_menu[1]; gook.append(gook_tmp)
    banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
    banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
    banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

    if i < 1067:
#        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
        side_tmp = day_menu[-2]; side.append(side_tmp)
    else: 
#        kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
        side_tmp  = day_menu[-1]; side.append(side_tmp)

### 점심 test set 제작

In [11]:
test_ln = x_test
#test_ln['bob'] = bob
test_ln['gook'] = gook
test_ln['banchan1'] = banchan1
test_ln['banchan2'] = banchan2 
test_ln['banchan3'] = banchan3
#test_ln['kimchi'] = kimchi 
test_ln['side'] = side

#test_ln['bob'] = test_ln['bob'].astype(object)
#test_ln['gook'] = test_ln['gook'].astype(object)
#test_ln['banchan1'] = test_ln['banchan1'].astype(object)
#test_ln['banchan2'] = test_ln['banchan2'].astype(object)
#test_ln['banchan3'] = test_ln['banchan3'].astype(object)
#test_ln['kimchi'] = test_ln['kimchi'].astype(object)
#test_ln['side'] = test_ln['side'].astype(object)

<ipython-input-11-dcbbd87f95cf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_ln['gook'] = gook
<ipython-input-11-dcbbd87f95cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_ln['banchan1'] = banchan1
<ipython-input-11-dcbbd87f95cf>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [12]:
len(x1_train)

1205

여기부턴 석식계

### train 저녁 전처리

In [13]:
# 일별 점심메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch) 만들기 
dinner1 = []
for day in range(len(train)):
    tmp = train.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    search = ')'   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    dinner1.append(tmp) 

### train 저녁메뉴 구분

In [14]:
# lunch train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []
for i, day_menu in enumerate(dinner1):
    if len(day_menu) >5:
#        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)
#        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
    else:
        bob.append(' '); gook.append(' '); banchan1.append(' ')
        banchan2.append(' '); banchan3.append(' '); kimchi.append(' ')

### 저녁 train set 제작

In [15]:
train_dn = x2_train
#train_dn['bob'] = bob
train_dn['gook'] = gook
train_dn['banchan1'] = banchan1; 
train_dn['banchan2'] = banchan2; 
train_dn['banchan3'] = banchan3
#train_dn['kimchi'] = kimchi 

#train_dn['bob'] = train_dn['bob'].astype(object)
#train_dn['gook'] = train_dn['gook'].astype(object)
#train_dn['banchan1'] = train_dn['banchan1'].astype(object)
#train_dn['banchan2'] = train_dn['banchan2'].astype(object)
#train_dn['banchan3'] = train_dn['banchan3'].astype(object)
#train_dn['kimchi'] = train_dn['kimchi'].astype(object)

<ipython-input-15-636c4f3b98d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dn['gook'] = gook
<ipython-input-15-636c4f3b98d2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dn['banchan1'] = banchan1;
<ipython-input-15-636c4f3b98d2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

### test 저녁 전처리

In [16]:
# 일별 저녁메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch) 만들기  test
dinner2 = []
for day in range(len(test)):
    tmp = train.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    search = ')'   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    dinner2.append(tmp) 

### test 저녁메뉴 구분

In [17]:
# lunch train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []
for i, day_menu in enumerate(dinner2):
    if len(day_menu) >5:
#        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)
#        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
    else:
        bob.append(' '); gook.append(' '); banchan1.append(' ')
        banchan2.append(' '); banchan3.append(' '); kimchi.append(' ')

### 저녁 test set 제작

In [18]:
test_dn = x_test
#test_dn['bob'] = bob
test_dn['gook'] = gook
test_dn['banchan1'] = banchan1; 
test_dn['banchan2'] = banchan2; 
test_dn['banchan3'] = banchan3
#test_dn['kimchi'] = kimchi 

#test_dn['bob'] = test_dn['bob'].astype(object)
#test_dn['gook'] = test_dn['gook'].astype(object)
#test_dn['banchan1'] = test_dn['banchan1'].astype(object)
#test_dn['banchan2'] = test_dn['banchan2'].astype(object)
#test_dn['banchan3'] = test_dn['banchan3'].astype(object)
#test_dn['kimchi'] = test_dn['kimchi'].astype('category')

<ipython-input-18-b9bdac183eee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dn['gook'] = gook
<ipython-input-18-b9bdac183eee>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dn['banchan1'] = banchan1;
<ipython-input-18-b9bdac183eee>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [19]:
print(len(train_ln),len(y1_train),len(test_ln))
print(len(train_dn),len(y2_train),len(test_dn))

1205 1205 50
1205 1205 50


In [20]:
categorical_features_indices_ln = np.where(train_ln != np.float)[0]
categorical_features_indices_dn = np.where(train_dn != np.float)[0]

<ipython-input-20-7009ca8652df>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices_ln = np.where(train_ln != np.float)[0]
<ipython-input-20-7009ca8652df>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices_dn = np.where(train_dn != np.float)[0]


In [21]:
(categorical_features_indices_ln)[600]

50

In [22]:
train_ln.dtypes

요일                  int64
본사시간외근무명령서승인건수      int64
식사가능자수            float64
월                   int64
일                   int64
주                   int32
년                   int64
gook               object
banchan1           object
banchan2           object
banchan3           object
side               object
dtype: object

In [25]:
train.shape

(1205, 17)

In [23]:
model1 = cat
model2 = cat

model1.fit(train_ln, y1_train, cat_features=categorical_features_indices_ln) #중식계
pred1 = model1.predict(test_ln)

model2.fit(train_dn, y2_train, cat_features=categorical_features_indices_dn) #석식계
pred2 = model2.predict(test_dn)

submission['중식계'] = pred1
submission['석식계'] = pred2

submission.to_csv('catA.csv', index=False)

CatBoostError: Invalid cat_features[144] = 12 value: index must be < 12.

---

## 데이터 로드

In [2]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

## 데이터

In [3]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [4]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [53]:
submission.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


In [54]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   object 
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   object 
 8   중식메뉴            1205 non-null   object 
 9   석식메뉴            1205 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 113.1+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              50 non-null     object 
 1   요일              50 non-null     object 
 2   본사정원수           50 non-null     int64  
 3   본사휴가자수          50 non-null     int64  
 4   본사출장자수          50 non-null     int64  
 5   본사시간외근무명령서승인건수  50 non-null     int64  
 6   현본사소속재택근무자수     50 non-null     float64
 7   조식메뉴            50 non-null     object 
 8   중식메뉴            50 non-null     object 
 9   석식메뉴            50 non-null     object 
dtypes: float64(1), int64(4), object(5)
memory usage: 4.0+ KB


In [8]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      50 non-null     object
 1   중식계     50 non-null     int64 
 2   석식계     50 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ KB


## `월, 일을 숫자형으로 변환`

In [3]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

print(train['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

0      2016-02-01
1      2016-02-02
2      2016-02-03
3      2016-02-04
4      2016-02-05
          ...    
1200   2021-01-20
1201   2021-01-21
1202   2021-01-22
1203   2021-01-25
1204   2021-01-26
Name: 일자, Length: 1205, dtype: datetime64[ns]



## `식사가능자수 , 식사참여율`

In [4]:
train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

In [11]:
#train['중식참여율'] = train['중식계'] / train['식사가능자수']
#train['석식참여율'] = train['석식계'] / train['식사가능자수']

#test['중식참여율'] = test['중식계'] / test['식사가능자수']
#test['석식참여율'] = test['석식계'] / test['식사가능자수']

## 수치 데이터만 피쳐로 선택

In [12]:
#x_train = train[['요일', '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]
#y1_train = train['중식계']
#y2_train = train['석식계']

#x_test = test[['요일', '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]

In [5]:
x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월','일','주','년']] #중식계
x2_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월','일','주','년']] #석식계


y1_train = train['중식계']
y2_train = train['석식계']

x_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월', '일','주','년']]

In [12]:
train.dtypes

일자                datetime64[ns]
요일                         int64
본사정원수                      int64
본사휴가자수                     int64
본사출장자수                     int64
본사시간외근무명령서승인건수             int64
현본사소속재택근무자수              float64
조식메뉴                      object
중식메뉴                      object
석식메뉴                      object
중식계                      float64
석식계                      float64
년                          int64
월                          int64
일                          int64
주                          int32
식사가능자수                   float64
dtype: object

In [11]:
x1_train

,요일,본사시간외근무명령서승인건수,식사가능자수,월,일,주,년
0,0,238,2551.0,2,1,5,2016
1,1,319,2551.0,2,2,5,2016
2,2,111,2545.0,2,3,5,2016
3,3,355,2497.0,2,4,5,2016
4,4,34,2323.0,2,5,5,2016
...,...,...,...,...,...,...,...
1200,2,4,2517.0,1,20,3,2021
1201,3,462,2540.0,1,21,3,2021
1202,4,1,2425.0,1,22,3,2021
1203,0,616,2549.0,1,25,4,2021


In [25]:
y1_train

0       1039.0
1        867.0
2       1017.0
3        978.0
4        925.0
         ...  
1200    1093.0
1201     832.0
1202     579.0
1203    1145.0
1204    1015.0
Name: 중식계, Length: 1205, dtype: float64

In [9]:
# check = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '중식계', '석식계', '월', '일']]

# plt.figure(figsize=(10,10)) # 도화지 크기를 키워줌
# sns.heatmap(check.corr(), annot = True) # seaborn의 heatmap 함수에 train.corr() 상관계수를 넣어준다.
                                        # annot < 수치도 같이 나타내줌

## 점심과 저녁을 식수 인원을 예측할 2개 모델 생성

In [14]:
model1 = cat
model2 = cat

## 학습

In [13]:
model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x_test)

Learning rate set to 0.040347
0:	learn: 204.1526956	total: 170ms	remaining: 2m 49s
1:	learn: 199.6481701	total: 173ms	remaining: 1m 26s
2:	learn: 194.7671381	total: 175ms	remaining: 58s
3:	learn: 190.2296638	total: 178ms	remaining: 44.2s
4:	learn: 185.7378701	total: 180ms	remaining: 35.8s
5:	learn: 181.6912133	total: 182ms	remaining: 30.1s
6:	learn: 177.9033157	total: 184ms	remaining: 26.1s
7:	learn: 174.0693282	total: 186ms	remaining: 23.1s
8:	learn: 170.3924669	total: 189ms	remaining: 20.8s
9:	learn: 166.9378433	total: 191ms	remaining: 18.9s
10:	learn: 164.1279138	total: 193ms	remaining: 17.4s
11:	learn: 160.9475931	total: 195ms	remaining: 16.1s
12:	learn: 158.1045127	total: 197ms	remaining: 14.9s
13:	learn: 155.2477891	total: 199ms	remaining: 14s
14:	learn: 152.4850589	total: 200ms	remaining: 13.2s
15:	learn: 149.9560140	total: 202ms	remaining: 12.4s
16:	learn: 147.8233391	total: 205ms	remaining: 11.8s
17:	learn: 145.5479629	total: 207ms	remaining: 11.3s
18:	learn: 143.2818583	total

In [13]:
model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x_test)

Learning rate set to 0.040347
0:	learn: 136.3178283	total: 4.37ms	remaining: 4.37s
1:	learn: 133.8488719	total: 6.5ms	remaining: 3.25s
2:	learn: 131.6634345	total: 10.3ms	remaining: 3.41s
3:	learn: 129.0973480	total: 12.9ms	remaining: 3.21s
4:	learn: 126.9072610	total: 15.6ms	remaining: 3.1s
5:	learn: 124.7395471	total: 18.7ms	remaining: 3.09s
6:	learn: 122.8741620	total: 21.3ms	remaining: 3.03s
7:	learn: 121.3647732	total: 23.9ms	remaining: 2.96s
8:	learn: 119.5439579	total: 26.3ms	remaining: 2.9s
9:	learn: 117.5951067	total: 29ms	remaining: 2.88s
10:	learn: 116.0784774	total: 31.1ms	remaining: 2.8s
11:	learn: 114.2476690	total: 33.3ms	remaining: 2.74s
12:	learn: 112.6021946	total: 35.7ms	remaining: 2.71s
13:	learn: 111.1504865	total: 37.9ms	remaining: 2.67s
14:	learn: 109.6355383	total: 41.1ms	remaining: 2.7s
15:	learn: 108.4026566	total: 43.8ms	remaining: 2.69s
16:	learn: 107.0219414	total: 48.5ms	remaining: 2.8s
17:	learn: 105.8099145	total: 50.6ms	remaining: 2.76s
18:	learn: 104.7

In [ ]:
submission['중식계'] = pred1
submission['석식계'] = pred2

In [20]:
submission.to_csv('cat.csv', index=False)

# ` 여기 아래부터 KFold 까지 적용. But Catboost인데 Kfold까진 과한듯 `

In [15]:
from sklearn.model_selection import KFold

In [16]:
k_fold = KFold(n_splits = 5, shuffle = True, random_state = 777)

In [17]:
model1 = cat
models1 = [] # 교차검증을 위한 바구나 5개

for train_idx, val_idx in k_fold.split(x1_train):
    x1_t = x1_train.iloc[train_idx]
    y1_t = y1_train.iloc[train_idx]
    x1_val = x1_train.iloc[val_idx]
    y1_val = y1_train.iloc[val_idx]
    
    models1.append(model1.fit(x1_t, y1_t, eval_set = (x1_val, y1_val), early_stopping_rounds=100, verbose = 100))

preds1 = []
for model1 in models1:
    preds1.append(model1.predict(x_test))    


model2 = cat
models2 = [] # 교차검증을 위한 바구나 5개

for train_idx, val_idx in k_fold.split(x2_train):
    x2_t = x2_train.iloc[train_idx]
    y2_t = y2_train.iloc[train_idx]
    x2_val = x2_train.iloc[val_idx]
    y2_val = y2_train.iloc[val_idx]
    
    models2.append(model2.fit(x2_t, y2_t, eval_set = (x2_val, y2_val), early_stopping_rounds=100, verbose = 100))
    

preds2 = []
for model2 in models2:
    preds2.append(model2.predict(x_test))

Learning rate set to 0.045759
0:	learn: 203.6156816	test: 203.6843745	best: 203.6843745 (0)	total: 1.65ms	remaining: 1.64s
100:	learn: 90.0358880	test: 105.7365497	best: 105.7365497 (100)	total: 158ms	remaining: 1.4s
200:	learn: 76.3781396	test: 100.3607728	best: 100.2926848 (195)	total: 312ms	remaining: 1.24s
300:	learn: 67.7743405	test: 100.0620399	best: 99.6946764 (266)	total: 471ms	remaining: 1.09s
400:	learn: 61.1937438	test: 99.5476335	best: 99.5476335 (400)	total: 628ms	remaining: 938ms
500:	learn: 55.6372236	test: 100.0226275	best: 99.5476335 (400)	total: 802ms	remaining: 798ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 99.54763354
bestIteration = 400

Shrink model to first 401 iterations.
Learning rate set to 0.045759
0:	learn: 201.9758810	test: 209.4369061	best: 209.4369061 (0)	total: 1.72ms	remaining: 1.72s
100:	learn: 90.1436269	test: 99.3719510	best: 99.3719510 (100)	total: 157ms	remaining: 1.4s
200:	learn: 75.9283323	test: 91.1811128	best: 91.18111

In [18]:
pred1 = np.mean(preds1, axis=0)
pred2 = np.mean(preds2, axis=0)

## 추론

In [73]:
submission['중식계'] = pred1
submission['석식계'] = pred2

## 결과 제출

In [74]:
submission.to_csv('catKfold.csv', index=False)

In [60]:
# 여태까지 최고성적은 요일, 일, 월만 추가한 것이었음.